#### DC Crime Stats
#### Twitter @SebbyStats
#### https://github.com/asranasinghe
<br>

**OVERVIEW**

This notebook charts Washington D.C. crime data and analyses potential causes.

Data pulled on January 14th, 2024 from the DC Metropolitan Police Department (MPD). See below:

* __[DC Crime Card](https://crimecards.dc.gov/all:crimes/all:weapons/dated::01012010:01012024/citywide:heat)__
* __[MPD Staffing Reports](https://mpdc.dc.gov/node/1653316)__

NOTE: Monthly MPD Staffing Reports were only available beginning October 2017. Staffing counts for prior months are only available on a FY basis. MPD FY seems to run from OctYYYY through SepYYYY. 

In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt

#### Load Data + Prep

In [76]:
# Load crime data, downloaded from DC Crime Card

crime = pd.read_csv('dc-crimes-search-results.csv')

# Violent crime only
crime = crime.loc[crime['offensegroup'] == 'violent']

# Extract start month, year
crime['month'] = pd.to_datetime(crime.START_DATE).dt.month
crime['year'] = pd.to_datetime(crime.START_DATE).dt.year

crime = crime.loc[crime['year'] >= 2010]


print(crime.shape)
print(crime.columns)
crime.head()


(74549, 31)
Index(['NEIGHBORHOOD_CLUSTER', 'CENSUS_TRACT', 'offensegroup', 'LONGITUDE',
       'END_DATE', 'offense-text', 'DISTRICT', 'YBLOCK', 'SHIFT', 'WARD',
       'YEAR', 'offensekey', 'BID', 'sector', 'PSA', 'ucr-rank', 'BLOCK_GROUP',
       'VOTING_PRECINCT', 'XBLOCK', 'BLOCK', 'START_DATE', 'CCN', 'OFFENSE',
       'OCTO_RECORD_ID', 'ANC', 'REPORT_DAT', 'METHOD', 'location', 'LATITUDE',
       'month', 'year'],
      dtype='object')


,NEIGHBORHOOD_CLUSTER,CENSUS_TRACT,offensegroup,LONGITUDE,END_DATE,offense-text,DISTRICT,YBLOCK,SHIFT,WARD,...,CCN,OFFENSE,OCTO_RECORD_ID,ANC,REPORT_DAT,METHOD,location,LATITUDE,month,year
0,cluster 25,10603.0,violent,-77.004738,"9/7/2018, 1:30:49 AM",robbery,5.0,137184.0,evening,6.0,...,18150624,robbery,NaN,6C,"9/7/2018, 5:59:29 PM",gun,"38.9025105195,-77.004738254",38.902511,9.0,2018.0
1,cluster 8,4802.0,violent,-77.017201,"8/16/2018, 3:55:52 PM",assault w/dangerous weapon,3.0,137407.0,evening,2.0,...,18136297,assault w/dangerous weapon,NaN,6E,"8/16/2018, 6:19:32 PM",others,"38.9045182119,-77.0172011533",38.904518,8.0,2018.0
2,cluster 33,9904.0,violent,-76.933983,NaN,assault w/dangerous weapon,6.0,134985.0,day,7.0,...,18076908,assault w/dangerous weapon,NaN,7E,"5/13/2018, 9:02:37 AM",knife,"38.8826825514,-76.9339825323",38.882683,5.0,2018.0
3,cluster 37,7407.0,violent,-76.981531,NaN,assault w/dangerous weapon,7.0,132146.0,day,8.0,...,18116129,assault w/dangerous weapon,NaN,8B,"7/15/2018, 1:54:34 PM",knife,"38.8571248848,-76.9815314071",38.857125,7.0,2018.0
4,cluster 37,7407.0,violent,-76.979032,"8/3/2018, 6:00:35 PM",assault w/dangerous weapon,7.0,131896.0,evening,8.0,...,18128504,assault w/dangerous weapon,NaN,8B,"8/3/2018, 6:08:53 PM",gun,"38.8548723559,-76.9790319536",38.854872,8.0,2018.0


In [90]:
# Load Monthly Staffing 

staff = pd.read_csv('mpd_staffing.csv')
staff['month'] = pd.to_datetime(staff.month).dt.date

staff.head()

,month,sworn_total,st_alt,st_adj,st_alt_src
0,2024-01-01,3342.0,NaN,3342,NaN
1,2023-12-01,3332.0,NaN,3332,NaN
2,2023-11-01,3333.0,NaN,3333,NaN
3,2023-10-01,3335.0,NaN,3335,NaN
4,2023-09-01,3328.0,NaN,3328,NaN


#### Analyze

In [87]:
# Get monthy violent crime counts

crime_agg = crime[['month', 'year']].value_counts().reset_index()
crime_agg.columns = ['month', 'year', 'crime_ct']

crime_agg['month'] = crime_agg.apply(lambda x: dt.date(int(x.year), int(x.month), 1), axis = 1)
crime_agg = crime_agg[['month', 'crime_ct']].sort_values(by = 'month')

crime_agg.head()

,month,crime_ct
70,2010-01-01,468
136,2010-02-01,331
57,2010-03-01,506
47,2010-04-01,538
11,2010-05-01,625


In [93]:
# Merge with staff count

final = pd.merge(crime_agg, staff, how = 'inner', on = 'month')
final = final[['month', 'crime_ct', 'st_adj']]
final.head()

,month,crime_ct,st_adj
0,2010-01-01,468,3961
1,2010-02-01,331,3961
2,2010-03-01,506,3961
3,2010-04-01,538,3961
4,2010-05-01,625,3961


,month,crime_ct
70,2010-01-01,468
136,2010-02-01,331
57,2010-03-01,506
47,2010-04-01,538
11,2010-05-01,625
